# The cars dataset - Cleaning

The cars dataset is a basic dataset of some cars and their mileage. Many versions of this dataset are available, we'll be using the one R has installed by default.

In this notebook we'll import the dataset and clean it up.

In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/mjochen/Beobank_course_material/main/Data_science/5%20Pandas%20introduction/Notebooks/files/mpg.csv")  
    
df.head(10) 

It was imported fine, but the first column is all wrong: the CSV has an index, but that index was seen as an extra column (and another index was added).

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/mjochen/Beobank_course_material/main/Data_science/5%20Pandas%20introduction/Notebooks/files/mpg.csv", index_col=0) 
df.head(10)

But what does all this mean? When looking at data there are a number of ways in which data can be dirty:

- Bad data (missing observations, dual observations, ...)
- Wrong structure (fields joined or spread out, ...)
- Dirty (wrong datatypes, string processing needed, ...)

The mpg-dataset has no bad data, and the structure is also fine. But there still are improvements.

By the way, what all the fields stand for is also well [explained](https://www.rdocumentation.org/packages/ggplot2/versions/3.3.6/topics/mpg) in the R-documentation.

Let's start with miles per gallon (cty and hwy for city and highway respectively). In liters per 100km that would be:

Liters100km = 	(100 * 3.785411784)/(1.609344 * MPG)

In [ ]:
df['clkm'] = [ (100 * 3.785411784)/(1.609344 * mpg) for mpg in df['cty']]
df['hwlkm'] = [ (100 * 3.785411784)/(1.609344 * mpg) for mpg in df['hwy']]

df.head(10)

The class of a car is actually a categorical value. This means it can only have a number of discrete values. Let's convert the class of the car to that type.

In [ ]:
df["class"] = pd.Categorical(df['class'])

The class of a car is not an ordered type of category but some are. Examples would be health labels on food, year a student is in, ... Let's assume it is though.

In [ ]:
df['class'].unique()

In [ ]:
from pandas.api.types import CategoricalDtype

# categories-list copied en rearranged from unique values
# and added a category just for fun
cat_type = CategoricalDtype(categories=['three wheeled car','2seater', 'subcompact', 'compact', 'midsize', 'minivan', 'suv', 'pickup'], ordered=True)

df["class"] = df['class'].astype(cat_type)

df.head()

And why are we doing this? Well, let's look at the result of a group by now...

In [ ]:
df.groupby('class').describe()

Some selections! Let's get all cars with an engine displacement of 3 or less.

In [ ]:
df[ df.displ <= 3]

And from this, only show the manufacturer and the nr of cylinders.

In [ ]:
df[ df.displ <= 3][['manufacturer', 'cyl']]

All cars having an odd number of cylinders or a displacement of exactly 2.8.

In [ ]:
df[ (df.displ == 2.8) | (df.cyl % 2 == 1)][['manufacturer', 'cyl', 'displ']]

Same as above, but sort by ascending number of cylinders.

In [ ]:
df[ (df.displ == 2.8) | (df.cyl % 2 == 1)][['manufacturer', 'cyl', 'displ']].sort_values('cyl')

What is the average displacement of all our cars? [Many options!](https://medium.com/analytics-vidhya/how-to-summarize-data-with-pandas-2c9edffafbaf)

info(): provides a concise summary of a dataframe.

In [ ]:
df.info()

describe(): Generates descriptive statistics that will provide visibility of the dispersion and shape of a dataset’s distribution. It excludes NaN values. It can be used for dataframe or a specific series.

In [ ]:
df.describe()

value_counts(): returns counts of unique values for the specified series. NaN values are excluded by default.

In [ ]:
df.displ.value_counts()

nunique(): Count distinct observations. Can be used for a dataframe or a series. By default, it exclude the NaN values.

In [ ]:
df.nunique()

sum(): Return the sum of the values for the requested axis. You can use it for both dataframe and series.

Watch out! Avoid selecting categorical columns to avoid a deprecated-error.

In [ ]:
df[['displ', 'cyl', 'hwy','model']].sum()

# the error:
# df[['displ','class']].sum()

count(): Return number of non-NA/null observations.

In [ ]:
df.count( numeric_only = True)

Min and Max, Mean and Median:

* min(): Return the minimum value
* max(): Return the maximum value
* mean(): Return the mean of the values
* median(): Return the median of the values

These functions can be applied to both dataframe and series.

(Note the class! The max is the last value of our ordered class.)

In [ ]:
df.max()

agg(): apply more than one aggregation operations to the same dataset over the specified axis.

In [ ]:
df[['displ', 'cyl', 'hwy','model']].agg(['count','min','max'])

groupby(): allows you to group data (by applying aggregate functions like sum, max, min…) with the same values into summary rows.

In [ ]:
df.groupby('class').cyl.mean()